# Data Transformation of Table "Faixa de Doações e Heranças"

## Environment Configuration

In [0]:
from pyspark.sql.utils import AnalysisException
from src.utils.udfs import functions_for_df_structure_management as ffdsm

## Data Ingestion from Bronze Layer

In [0]:
df_donation_range_and_inheritance = spark.table("brazilian_tax_big_numbers.bronze_layer.delta_faixa_de_doacoes_e_herancas")

In [0]:
display(df_donation_range_and_inheritance)

## Data Transformation

In [0]:
df_donation_range_and_inheritance = df_donation_range_and_inheritance.dropna(how='all')

In [0]:
df_casted_donation_range_and_inheritance = ffdsm.cast_columns_to_float(df_donation_range_and_inheritance, ["AnoCalendario", "FaixaDeDoacoesEHerancas"])

In [0]:
df_casted_donation_range_and_inheritance = ffdsm.rename_columns_with_df_name(df_casted_donation_range_and_inheritance, "FaixaDeDoacoesEHerancas", ["AnoCalendario", "FaixaDeDoacoesEHerancas"])

In [0]:
dbutils.data.summarize(df_casted_donation_range_and_inheritance)

In [0]:
df_filled_donation_range_and_inheritance = ffdsm.fill_nulls(df_casted_donation_range_and_inheritance, ["AnoCalendario", "FaixaDeDoacoesEHerancas"])

## Save as Delta in Silver Layer

In [0]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS brazilian_tax_big_numbers.silver_layer")

In [0]:
error = None

try:
    df_filled_donation_range_and_inheritance.write \
        .mode("overwrite") \
        .saveAsTable(f"brazilian_tax_big_numbers.silver_layer.delta_faixa_de_doacoes_e_herancas")
    error = None
except Exception as e:
    error = str(e)
    print(error)